In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TransformerModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 72)
    output_chunk_length = trial.suggest_int("output_chunk_length", 1, 24)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 64, 256)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    d_model = trial.suggest_categorical('d_model', [64, 128, 256])
    nhead = trial.suggest_categorical('nhead', [2, 4, 8])
    num_encoder_layers = trial.suggest_int('num_encoder_layers', 1, 4)
    num_decoder_layers = trial.suggest_int('num_decoder_layers', 1, 4)
    activation = trial.suggest_categorical('activation', ["GLU", "Bilinear", "ReGLU", "GEGLU", "SwiGLU", "ReLU", "GELU"])

    # Try different loss functions
    loss_functions = {
        'mse': torch.nn.MSELoss(),
        'mae': torch.nn.L1Loss(),
        'huber': torch.nn.HuberLoss(delta=1.0),
        'smoothl1': torch.nn.SmoothL1Loss()
    }
    loss_name = trial.suggest_categorical("loss_fn", list(loss_functions.keys()))

    # Try different optimizers
    optimizer_names = ['Adam', 'AdamW', 'RMSprop', 'SGD']
    optimizer_name = trial.suggest_categorical("optimizer", optimizer_names)
    optimizer_dict = {
        'Adam': torch.optim.Adam,
        'AdamW': torch.optim.AdamW,
        'RMSprop': torch.optim.RMSprop,
        'SGD': torch.optim.SGD
    }

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }
    n = int(len(df) * 0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")
    print(f"Validation set: {len(val_fold)} samples")

    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"
    _model_name = "model_optuna_temp"
    os.makedirs(_work_dir, exist_ok=True)

    # Additional optimizer kwargs based on optimizer type
    opt_kwargs = {"lr": lr, "weight_decay": weight_decay}
    if optimizer_name == 'SGD':
        opt_kwargs['momentum'] = 0.9

    model = TransformerModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        activation=activation,
        n_epochs=30,
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=loss_functions[loss_name],
        optimizer_cls=optimizer_dict[optimizer_name],
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 5, "min_lr": 1e-7},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True,
        optimizer_kwargs=opt_kwargs,
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=False,
        dataloader_kwargs={"num_workers": 11},
    )

    try:
        loaded_model = TransformerModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using the in-memory trained model.")
        loaded_model = model

    forecasts = loaded_model.historical_forecasts(
        val_scaled,
        forecast_horizon=output_chunk_length,
        stride=1,
        retrain=False,
        verbose=False
    )

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(val_scaled)
    metrics = {}
    print("Starting time series verification for consistency...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_mae_val = mae(val_scaled, forecasts)
    print(f"The MAE for this fold was {overall_mae_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "d_model": d_model,
        "nhead": nhead,
        "num_encoder_layers": num_encoder_layers,
        "num_decoder_layers": num_decoder_layers,
        "activation": activation,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "loss_function": loss_name,
        "optimizer": optimizer_name,
        "overall_mae_val": overall_mae_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return overall_mae_val if not np.isnan(overall_mae_val) else float("inf")


In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 13
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-26 10:54:04,281] A new study created in memory with name: no-name-2576eeaa-228a-4bea-a174-5f34a69d9d37



Starting Trial 0
Hyperparameters: {'input_chunk_length': 40, 'output_chunk_length': 22, 'dropout': 0.3445176621008879, 'batch_size': 136, 'lr': 3.200692693553335e-06, 'weight_decay': 0.002240791607783318, 'd_model': 256, 'nhead': 4, 'num_encoder_layers': 2, 'num_decoder_layers': 2, 'activation': 'ReGLU', 'loss_fn': 'huber', 'optimizer': 'SGD'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 0.294
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.294
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.293
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.292
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.292
Metric val_l

Model loaded from checkpoint for trial 0


[I 2025-05-26 10:56:54,608] Trial 0 finished with value: 0.8126001880062268 and parameters: {'input_chunk_length': 40, 'output_chunk_length': 22, 'dropout': 0.3445176621008879, 'batch_size': 136, 'lr': 3.200692693553335e-06, 'weight_decay': 0.002240791607783318, 'd_model': 256, 'nhead': 4, 'num_encoder_layers': 2, 'num_decoder_layers': 2, 'activation': 'ReGLU', 'loss_fn': 'huber', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.8126001880062268.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
H

...

Starting time series verification for consistency...

Performance metrics:
                                    MSE        RMSE         MAE           R2  \
Temperature                    6.883532    2.623649    2.081916     0.773918   
Precipitation_accumulated   4934.344109   70.244887   62.746625 -1436.726346   
Humidity                      58.717085    7.662707    5.923078     0.599289   
Wind_Speed_kmh                 1.080019    1.039240    0.815715     0.199795   
Soil_Moisture              32327.759375  179.799220  153.992007     0.434981   
Soil_Temperature               4.091778    2.022814    1.649416     0.746158   
Wind_Dir_Sin                   0.517373    0.719287    0.637044     0.060196   
Wind_Dir_Cos                   0.389919    0.624435    0.515682    -0.063398   

                                SMAPE  
Temperature                 33.969020  
Precipitation_accumulated   14.751211  
Humidity                     7.023493  
Wind_Speed_kmh              70.479078  
Soil

/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 0.033
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 0.030
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 0.027
Metric val_loss improved by 0.006 >= min_delta = 0.0005. New best score: 0.022
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 0.018
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.017
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 

Model loaded from checkpoint for trial 90


[I 2025-05-26 13:02:55,965] Trial 90 finished with value: 0.11596847552058936 and parameters: {'input_chunk_length': 27, 'output_chunk_length': 3, 'dropout': 0.12746206490265272, 'batch_size': 136, 'lr': 0.0026536697121054705, 'weight_decay': 0.0001899286237956314, 'd_model': 128, 'nhead': 2, 'num_encoder_layers': 1, 'num_decoder_layers': 1, 'activation': 'Bilinear', 'loss_fn': 'huber', 'optimizer': 'Adam'}. Best is trial 88 with value: 0.10533275687549581.


Starting time series verification for consistency...

Performance metrics:
                                    MSE        RMSE         MAE          R2  \
Temperature                    5.230533    2.287036    1.829829    0.828209   
Precipitation_accumulated   1729.834682   41.591281   37.229365 -503.024211   
Humidity                      53.447228    7.310761    5.755345    0.635253   
Wind_Speed_kmh                 1.036295    1.017986    0.785449    0.232190   
Soil_Moisture              18635.759897  136.512856  107.540970    0.674287   
Soil_Temperature               5.742190    2.396287    2.007489    0.643771   
Wind_Dir_Sin                   0.517118    0.719109    0.619282    0.060660   
Wind_Dir_Cos                   0.373839    0.611423    0.506419   -0.019544   

                                SMAPE  
Temperature                 31.936431  
Precipitation_accumulated    8.425183  
Humidity                     6.755652  
Wind_Speed_kmh              69.854756  
Soil_Moisture

In [8]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.10533275687549581
Best Parameters: {'input_chunk_length': 28, 'output_chunk_length': 2, 'dropout': 0.11806248396557542, 'batch_size': 153, 'lr': 0.0007930360578326748, 'weight_decay': 0.00013997625135104947, 'd_model': 128, 'nhead': 4, 'num_encoder_layers': 1, 'num_decoder_layers': 1, 'activation': 'Bilinear', 'loss_fn': 'huber', 'optimizer': 'Adam'}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
